In [247]:
import pandas as pd
import numpy as np
import re

In [248]:
df = pd.read_json('../Data/output.json')

In [249]:
print(df.dtypes)
print(df.columns)

product_id                                                                                                                                                                                                                                                 object
Тип                                                                                                                                                                                                                                                        object
Льготный период                                                                                                                                                                                                                                            object
Кредитный лимит                                                                                                                                                                                                                   

In [250]:
df.loc[df['Тип'].isna(), "Тип"]=""

In [251]:
print(sorted(set(df["Тип"].unique())))
print(len(set(df["Тип"].unique())))

['', 'MasterСard Gold', 'MasterСard Gold, Мир Virtual', 'MasterСard Platinum', 'MasterСard Platinum/Unembossed', 'MasterСard Standard', 'MasterСard Standard/Unembossed', 'MasterСard World', 'MasterСard World, Мир Классическая', 'UnionPay Classic/Instant Issue, Мир Классическая/Моментальная', 'Visa Classic', 'Visa Classic, MasterСard Standard', 'Visa Classic, MasterСard World', 'Visa Classic/Gold', 'Visa Gold', 'Visa Gold/Platinum', 'Visa Infinite', 'Visa Platinum, MasterСard Platinum', 'Visa Signature', 'Visa Signature, MasterСard World', 'Мир Classic', 'Мир Platinum', 'Мир Supreme', 'Мир Virtual', 'Мир Классическая', 'Мир Классическая/Virtual']
26


In [252]:
df['Тип'] = df['Тип'].replace(regex={
    'Мир': 'Mir',
    'Классическая': 'Classic',
    'Моментальная': 'Momentum'
})
print(len(set(df["Тип"].unique())))

25


In [253]:
def determine_pay_system(card_type):
    if 'MasterСard' in card_type:
        return 'MasterСard'
    elif 'Visa' in card_type:
        return 'Visa'
    elif 'Mir' in card_type:
        return 'Mir'
    elif 'UnionPay' in card_type:
        return 'UnionPay'
    else:
        return None

In [254]:
df['paySystem'] = df['Тип'].apply(determine_pay_system)
print(df.head(5))

                             product_id          Тип Льготный период  \
0  4966ba76-928e-4ada-94e2-2048df61319b  Mir Classic         60 дней   
1  8b89c6f6-69fa-4142-835d-6b6b5077930e  Mir Classic        4 месяца   
2  19f65d42-4a94-40e5-817a-4a66ff1a0230  Mir Classic        до 3 лет   
3  b5b44bb8-95a7-4f7b-87a9-33fd7e082508  Mir Classic         55 дней   
4  5429d0db-a91b-4cff-8203-446fdae8d040  Mir Classic        120 дней   

  Кредитный лимит Обслуживание в первый год Обслуживание со второго года  \
0  до 1 000 000 ₽                 бесплатно                        990 ₽   
1  до 5 000 000 ₽                       NaN                          NaN   
2    до 500 000 ₽                       NaN                          NaN   
3    до 600 000 ₽                       NaN                          NaN   
4  до 2 000 000 ₽                       NaN                          NaN   

  Минимальный возраст для получения карты  \
0                                  18 лет   
1                   

In [255]:
categories = [
    'Такси', 'Кино и развлечения', 'Кафе и рестораны', 'Красота и здоровье', 
    'Одежда и обувь', 'Супермаркеты', 'Транспорт', 'Туры для путешествий', 
    'Бронирование отелей', 'Электроника и бытовая техника', 'Игры', 
    'Канцтовары и книги', 'Товары для животных', 'АЗС', 'Спорттовары', 
    'Дом и ремонт', 'Обучение', 'Каршеринг', 'Автоуслуги', 'Цветы', 
    'Аптеки', 'Детские товары', 'Авиабилеты', 'Ж/Д билеты', 'Фастфуд', 
    'Маркетплейсы', 'Театры', 'Оптика', 'Фитнес', 'Аренда автомобилей', 
    'Привет, Мир!', 'ЖКХ', 'На всё', 'У партнеров'
]

In [256]:
df['Cashback'] = ''


cashback_columns = [
    col for col in df.columns
    if (
        (col.startswith("За покупки") or "на всё" in col.lower()) and
        df[col].apply(lambda x: isinstance(x, str) and "%" in x).any()
    )
]


for col in cashback_columns:
    normalized_col = col.lower()
    
    extracted = []
    for category in categories:
        if category.lower() in normalized_col:
            extracted.append(category)
        if "у партнеров" in normalized_col and category == "У партнеров":
            extracted.append(category)
        if "на всё" in normalized_col and category == "На всё":
            extracted.append(category)
    
    extracted = list(set(extracted))
    
    if extracted:
        df['Cashback'] = df.apply(
            lambda row: (
                f"{row['Cashback']}; " + 
                "; ".join([f"{cat}: {row[col]}" for cat in extracted])
                if not pd.isna(row[col]) else row['Cashback']
            ),
            axis=1
        )


df['Cashback'] = df['Cashback'].str.strip('; ').replace('', pd.NA)


df = df[[
    col for col in df.columns
    if col not in cashback_columns and not col.startswith("За покупки")
]]

print(df)

                               product_id           Тип Льготный период  \
0    4966ba76-928e-4ada-94e2-2048df61319b   Mir Classic         60 дней   
1    8b89c6f6-69fa-4142-835d-6b6b5077930e   Mir Classic        4 месяца   
2    19f65d42-4a94-40e5-817a-4a66ff1a0230   Mir Classic        до 3 лет   
3    b5b44bb8-95a7-4f7b-87a9-33fd7e082508   Mir Classic         55 дней   
4    5429d0db-a91b-4cff-8203-446fdae8d040   Mir Classic        120 дней   
..                                    ...           ...             ...   
126  dc198090-1f85-4229-af6f-b57f1902744c   Mir Classic        200 дней   
127  ea090ad6-fdae-4c26-9f8e-51c5bf38d0b9   Mir Classic        115 дней   
128  f62aa11e-8b13-49ff-bf9b-f16cf9319cee   Mir Supreme        115 дней   
129  8468f42c-9eb1-45c9-b041-0dd5714eb0b1                      415 дней   
130  df665bbe-6fa4-40d4-9644-8c3c028b38be  Visa Classic         50 дней   

                Кредитный лимит Обслуживание в первый год  \
0                до 1 000 000 ₽       

In [257]:
print(df.columns)
print(df[['Cashback']].head(5))

Index(['product_id', 'Тип', 'Льготный период', 'Кредитный лимит',
       'Обслуживание в первый год', 'Обслуживание со второго года',
       'Минимальный возраст для получения карты',
       'Максимальный возраст для закрытия карты', 'Гражданство', 'Регистрация',
       'Стаж работы на последнем месте',
       'Условия снятия наличных: кредитные средства',
       'Условия снятия наличных: собственные средства', 'Максимальный',
       'Способ возврата', 'Максимальный возврат по кешбэку в месяц',
       'Обслуживание', 'Ограничения на снятие наличных: кредитные средства',
       '2-НДФЛ, 3-НДФЛ, 4-НДФЛ', 'Справка по форме банка',
       'Ограничения на снятие наличных: собственные средства',
       'Другие документы', 'Справка из пенсионного фонда',
       'Справка в свободной форме',
       'Документальное подтверждение дополнительного дохода', 'paySystem',
       'Cashback'],
      dtype='object')
                                            Cashback
0                                  У

In [258]:
df['Обслуживание в первый год'] = df['Обслуживание в первый год'].fillna(df['Обслуживание'])
df['Обслуживание со второго года'] = df['Обслуживание со второго года'].fillna(df['Обслуживание'])


df = df.drop(columns=['Обслуживание'])

In [259]:
def extract_cash_info(row):
    cmtc_in = cmtc_out = cmtc_b_in = cmtc_b_out = " "
    
    if isinstance(row['Условия снятия наличных: кредитные средства'], list):
        for item in row['Условия снятия наличных: кредитные средства']:
            commission = item.get("Комиссия", " ")
            amount = item.get("Сумма", " ")
            
            value = f"{commission} ({amount})" if amount != " " else commission
            
            if item.get("") == "В банкоматах банка":
                cmtc_in = value
            elif item.get("") == "В банкоматах других банков":
                cmtc_out = value
    
    if isinstance(row['Ограничения на снятие наличных: кредитные средства'], list):
        for item in row['Ограничения на снятие наличных: кредитные средства']:
            limit = item.get("Ограничение", " ")
            
            if item.get("") == "В банкоматах банка":
                cmtc_b_in = limit
            elif item.get("") == "В банкоматах других банков":
                cmtc_b_out = limit
    
    return pd.Series([cmtc_in, cmtc_out, cmtc_b_in, cmtc_b_out])


df[["CMTC_in", "CMTC_out", "CMTC_B_in", "CMTC_B_out"]] = df.apply(extract_cash_info, axis=1)

df = df.drop(columns=[
    "Условия снятия наличных: кредитные средства",
    "Ограничения на снятие наличных: кредитные средства"
])

In [260]:
def extract_cash_info(row):
    mmtc_in = mmtc_out = mmtc_b_in = mmtc_b_out = " "
    
    # Обработка условий снятия (комиссии)
    if isinstance(row['Условия снятия наличных: собственные средства'], list):
        for item in row['Условия снятия наличных: собственные средства']:
            commission = item.get("Комиссия", " ")
            amount = item.get("Сумма", " ")
            
            value = f"{commission} ({amount})" if amount != " " else commission
            
            if item.get("") == "В банкоматах банка":
                mmtc_in = value
            elif item.get("") == "В банкоматах других банков":
                mmtc_out = value
    
    # Обработка ограничений (лимиты)
    if isinstance(row['Ограничения на снятие наличных: собственные средства'], list):
        for item in row['Ограничения на снятие наличных: собственные средства']:
            limit = item.get("Ограничение", " ")
            
            if item.get("") == "В банкоматах банка":
                mmtc_b_in = limit
            elif item.get("") == "В банкоматах других банков":
                mmtc_b_out = limit
    
    return pd.Series([mmtc_in, mmtc_out, mmtc_b_in, mmtc_b_out])

df[["MMTC_in", "MMTC_out", "MMTC_B_in", "MMTC_B_out"]] = df.apply(extract_cash_info, axis=1)

df = df.drop(columns=[
    "Условия снятия наличных: собственные средства",
    "Ограничения на снятие наличных: собственные средства"
])

print(df.head(5))

                             product_id          Тип Льготный период  \
0  4966ba76-928e-4ada-94e2-2048df61319b  Mir Classic         60 дней   
1  8b89c6f6-69fa-4142-835d-6b6b5077930e  Mir Classic        4 месяца   
2  19f65d42-4a94-40e5-817a-4a66ff1a0230  Mir Classic        до 3 лет   
3  b5b44bb8-95a7-4f7b-87a9-33fd7e082508  Mir Classic         55 дней   
4  5429d0db-a91b-4cff-8203-446fdae8d040  Mir Classic        120 дней   

  Кредитный лимит Обслуживание в первый год Обслуживание со второго года  \
0  до 1 000 000 ₽                 бесплатно                        990 ₽   
1  до 5 000 000 ₽                 бесплатно                    бесплатно   
2    до 500 000 ₽                 бесплатно                    бесплатно   
3    до 600 000 ₽                 бесплатно                    бесплатно   
4  до 2 000 000 ₽                 бесплатно                    бесплатно   

  Минимальный возраст для получения карты  \
0                                  18 лет   
1                   

In [261]:
print(df['Справка из пенсионного фонда'].value_counts())
print(df['Справка в свободной форме'].value_counts())
print(df['Документальное подтверждение дополнительного дохода'].value_counts())
print(df['Другие документы'].value_counts())
print(df['2-НДФЛ, 3-НДФЛ, 4-НДФЛ'].value_counts())

Справка из пенсионного фонда
по выбору, если требуется один или несколько документов из одной группы    8
желательно или по требованию                                               1
обязательно                                                                1
Name: count, dtype: int64
Справка в свободной форме
по выбору, если требуется один или несколько документов из одной группы    3
Name: count, dtype: int64
Документальное подтверждение дополнительного дохода
по выбору, если требуется один или несколько документов из одной группы    1
Name: count, dtype: int64
Другие документы
по выбору, если требуется один или несколько документов из одной группы    6
обязательно                                                                2
желательно или по требованию                                               1
Name: count, dtype: int64
2-НДФЛ, 3-НДФЛ, 4-НДФЛ
по выбору, если требуется один или несколько документов из одной группы    44
обязательно                                            

In [262]:
df.columns

Index(['product_id', 'Тип', 'Льготный период', 'Кредитный лимит',
       'Обслуживание в первый год', 'Обслуживание со второго года',
       'Минимальный возраст для получения карты',
       'Максимальный возраст для закрытия карты', 'Гражданство', 'Регистрация',
       'Стаж работы на последнем месте', 'Максимальный', 'Способ возврата',
       'Максимальный возврат по кешбэку в месяц', '2-НДФЛ, 3-НДФЛ, 4-НДФЛ',
       'Справка по форме банка', 'Другие документы',
       'Справка из пенсионного фонда', 'Справка в свободной форме',
       'Документальное подтверждение дополнительного дохода', 'paySystem',
       'Cashback', 'CMTC_in', 'CMTC_out', 'CMTC_B_in', 'CMTC_B_out', 'MMTC_in',
       'MMTC_out', 'MMTC_B_in', 'MMTC_B_out'],
      dtype='object')

In [263]:
df['2-НДФЛ, 3-НДФЛ, 4-НДФЛ'] = df['2-НДФЛ, 3-НДФЛ, 4-НДФЛ'].fillna('нет требуется')
df['Справка в свободной форме'] =  df['Справка в свободной форме'].fillna('нет требуется')
df['Справка по форме банка'] =  df['Справка по форме банка'].fillna('нет требуется')
df['Справка из пенсионного фонда'] =  df['Справка из пенсионного фонда'].fillna('нет требуется')
df['Документальное подтверждение дополнительного дохода'] =  df['Документальное подтверждение дополнительного дохода'].fillna('нет требуется')
df['Другие документы'] =  df['Другие документы'].fillna('нет требуется')

In [264]:
df['Максимальный возврат по кешбэку в месяц'] =  df['Максимальный возврат по кешбэку в месяц'].fillna('')
df['Способ возврата'] =  df['Способ возврата'].fillna('валюта карты')
df['Максимальный'] = df['Максимальный'].fillna(
    "до " + 
    df['Cashback']
    .str.extractall(r'(\d+)')[0]
    .astype(int)
    .groupby(level=0)
    .max()
    .reindex(df.index, fill_value=0)
    .astype(str)
)

In [272]:
df.columns

Index(['product_id', 'Тип', 'Льготный период', 'Кредитный лимит',
       'Обслуживание в первый год', 'Обслуживание со второго года',
       'Минимальный возраст для получения карты',
       'Максимальный возраст для закрытия карты', 'Гражданство', 'Регистрация',
       'Стаж работы на последнем месте', 'Максимальный', 'Способ возврата',
       'Максимальный возврат по кешбэку в месяц', '2-НДФЛ, 3-НДФЛ, 4-НДФЛ',
       'Справка по форме банка', 'Другие документы',
       'Справка из пенсионного фонда', 'Справка в свободной форме',
       'Документальное подтверждение дополнительного дохода', 'paySystem',
       'Cashback', 'CMTC_in', 'CMTC_out', 'CMTC_B_in', 'CMTC_B_out', 'MMTC_in',
       'MMTC_out', 'MMTC_B_in', 'MMTC_B_out', 'product_name', 'bank',
       'reviews_count', 'rating'],
      dtype='object')

In [270]:
name_df = pd.read_json('../Data/name_card.json')

name_df = name_df[["product_id", "product_name", "bank", "reviews_count", "rating"]]
name_df["reviews_count"] = name_df["reviews_count"].replace("N/A", "0").astype(int)
name_df["rating"] = name_df["rating"].replace("N/A", "0")
name_df["rating"] = name_df["rating"].apply(lambda x: x.replace(',', '.')).astype(float)

df = df.merge(name_df, on='product_id', how='outer', suffixes=('', '_new'))
print(df.head(4))

for col in ["product_name", "bank", "reviews_count", "rating"]:
    df[col] = df[f"{col}"].combine_first(df[col])

df = df.reset_index(drop=True)

                             product_id          Тип Льготный период  \
0  01d0389d-f850-4352-a557-37ea6ddc4204  Mir Supreme         55 дней   
1  02acbe58-7aba-4aae-8577-9f4804bbd431                     180 дней   
2  02d82bb8-cbe0-4bb7-bb01-8f2808bf0c67  Mir Classic         51 день   
3  05bacd22-f37c-4219-a29e-18658123f836                     1094 дня   

              Кредитный лимит Обслуживание в первый год  \
0                       любой             490 ₽ в месяц   
1  от 10 000 ₽ до 1 000 000 ₽                 бесплатно   
2    от 20 000 ₽ до 100 000 ₽                 бесплатно   
3                до 150 000 ₽                 бесплатно   

  Обслуживание со второго года Минимальный возраст для получения карты  \
0                490 ₽ в месяц                                  18 лет   
1                    бесплатно                                  21 год   
2                    бесплатно                                  20 лет   
3                    бесплатно                 

In [ ]:
column_mapping = {
    # Русские названия → английские
    'Тип': 'type',
    'Льготный период': 'grace_period',
    'Кредитный лимит': 'credit_limit',
    'Обслуживание в первый год': 'first_year_fee',
    'Обслуживание со второго года': 'second_year_fee',
    'Минимальный возраст для получения карты': 'min_age',
    'Максимальный возраст для закрытия карты': 'max_age',
    'Гражданство': 'citizenship',
    'Регистрация': 'registration',
    'Стаж работы на последнем месте': 'work_experience',
    'Максимальный': 'maximum',
    'Способ возврата': 'repayment_method',
    'Максимальный возврат по кешбэку в месяц': 'max_monthly_cashback',
    '2-НДФЛ, 3-НДФЛ, 4-НДФЛ': 'ndfl_2_3_4',
    'Справка по форме банка': 'bank_certificate',
    'Другие документы': 'other_documents',
    'Справка из пенсионного фонда': 'pension_certificate',
    'Справка в свободной форме': 'free_form_certificate',
    'Документальное подтверждение дополнительного дохода': 'additional_income_proof',
    'paySystem': 'payment_system',
    'Cashback': 'cashback',
}


def normalize_column_name(name):
    return name.lower().replace(' ', '_').replace('-', '_')


new_columns = []
for col in df.columns:
    if col in column_mapping:
        new_col = column_mapping[col]
    else:
        new_col = normalize_column_name(col)
    new_columns.append(new_col)

df.columns = new_columns
print(df.columns.tolist())

Новые названия колонок:
['product_id', 'type', 'grace_period', 'credit_limit', 'first_year_fee', 'second_year_fee', 'min_age', 'max_age', 'citizenship', 'registration', 'work_experience', 'maximum', 'repayment_method', 'max_monthly_cashback', 'ndfl_2_3_4', 'bank_certificate', 'other_documents', 'pension_certificate', 'free_form_certificate', 'additional_income_proof', 'payment_system', 'cashback', 'cmtc_in', 'cmtc_out', 'cmtc_b_in', 'cmtc_b_out', 'mmtc_in', 'mmtc_out', 'mmtc_b_in', 'mmtc_b_out', 'product_name', 'bank', 'reviews_count', 'rating']


In [274]:
df.to_csv('data.csv', index=False, encoding='utf-8')
df.to_json('data.json', orient='records', force_ascii=False, indent=2)